# CS 109A/STAT 121A/AC 209A/CSCI E-109A: Homework 4
# Regularization, High Dimensionality, PCA


**Harvard University**<br/>
**Fall 2017**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine

---

### INSTRUCTIONS

- To submit your assignment follow the instructions given in canvas.
- Restart the kernel and run the whole notebook again before you submit. 
- Do not include your name(s) in the notebook even if you are submitting as a group. 
- If you submit individually and you have worked with someone, please include the name of your [one] partner below. 

---

In [16]:
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
%matplotlib inline
pd.set_option('display.max_rows', 500)

# Continuing Bike Sharing Usage Data

In this homework, we will focus on multiple linear regression, regularization, dealing with high dimensionality, and PCA. We will continue to build regression models for the Capital Bikeshare program in Washington D.C.  See Homework 3 for more information about the data.

*Note: please make sure you use all the processed data from HW 3 Part (a)...you make want to save the data set on your computer and reread the csv/json file here.


In [10]:
# open the files
train_data = pd.read_csv('Bikeshare_train.csv', sep=",", header=0)
test_data = pd.read_csv('Bikeshare_test.csv', sep=",", header=0)

In [11]:
train_data.rename(index=str, columns={"count": "rentals"}, inplace=True)
test_data.rename(index=str, columns={"count": "rentals"}, inplace=True)

In [ ]:
y_train = train_data['rentals'].values
X_train = train_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3']].values

y_test = test_data['rentals'].values
X_test = test_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3']].values
train_data = train_data.drop('Unnamed: 0', 1)
train_data = train_data.drop('Unnamed: 0.1', 1)

In [ ]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

print(X_train.shape, y_train.shape, X_test.shape , y_test.shape)

In [14]:
test_data.head(5)

,Unnamed: 0,season,month,holiday,day_of_week,workingday,weather,temp,atemp,humidity,windspeed,rentals
0,0,1.0,2.0,0.0,4.0,1.0,1.0,2.0,6.0,56.2083,0.194037,3830.0
1,1,1.0,12.0,0.0,4.0,1.0,2.0,1.0,3.0,65.2917,0.350133,2114.0
2,2,2.0,6.0,0.0,4.0,1.0,2.0,36.0,37.0,56.8333,0.149883,915.0
3,3,1.0,2.0,0.0,1.0,1.0,1.0,8.0,12.0,49.0833,0.268033,4322.0
4,4,2.0,5.0,0.0,0.0,0.0,1.0,28.0,29.0,69.7083,0.215171,6591.0


## Part (f): Regularization/Penalization Methods

As an alternative to selecting a subset of predictors and fitting a regression model on the subset, one can fit a linear regression model on all predictors, but shrink or regularize the coefficient estimates to make sure that the model does not "overfit" the training set. 

Use the following regularization techniques to fit linear models to the training set:
- Ridge regression
- Lasso regression
    
You may choose the shrikage parameter $\lambda$ from the set $\{10^{-5}, 10^{-4},...,10^{4},10^{5}\}$ using cross-validation. In each case, 

- How do the estimated coefficients compare to or differ from the coefficients estimated by a plain linear regression (without shrikage penalty) in Part (b) fropm HW 3? Is there a difference between coefficients estimated by the two shrinkage methods? If so, give an explantion for the difference.
- List the predictors that are assigned a coefficient value close to 0 (say < 1e-10) by the two methods. How closely do these predictors match the redundant predictors (if any) identified in Part (c) from HW 3?
- Is there a difference in the way Ridge and Lasso regression assign coefficients to the predictors `temp` and `atemp`? If so, explain the reason for the difference.

We next analyze the performance of the two shrinkage methods for different training sample sizes:
- Generate random samples of sizes 100, 150, ..., 400 from the training set. You may use the following code to draw a random sample of a specified size from the training set:

In [ ]:
alpha_values = [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4, 10**5]
ridge_models = {}
lasso_models = {}

for alpha_val in alpha_values:

    # build the ridge and lasso regression model with specified lambda, ie, alpha
    ridge_reg = Ridge(alpha = alpha_val)
    lasso_reg = Lasso(alpha = alpha_val)

    # cross validate
    scores_r = cross_val_score(ridge_reg, X_train, y_train, cv=10)
    score_r = np.mean(scores_r)
    temp_dict_r = {alpha_val : score_r}
    ridge_models.update(temp_dict_r)
    
    scores_l = cross_val_score(lasso_reg, X_train, y_train, cv=10)
    score_l = np.mean(scores_l)
    temp_dict_l = {alpha_val : score_l}
    lasso_models.update(temp_dict_l)



best_lasso_alpha = max(lasso_models, key=lasso_models.get)
best_lasso_score =lasso_models.get(best_lasso_alpha)

best_ridge_alpha = max(ridge_models, key=ridge_models.get)
best_ridge_score =ridge_models.get(best_ridge_alpha)

print('The best ridge model has alpha=', best_ridge_alpha)
print('The best ridge model has score=', best_ridge_score)

print('The best lasso model has alpha=', best_lasso_alpha)
print('The best lasso model has score=', best_lasso_score)

In [ ]:
# fit best validated Ridge and lasso models

ridge_reg = Ridge(alpha = best_ridge_alpha)
lasso_reg = Lasso(alpha = best_lasso_alpha)

#fit the model to training data
ridge_reg.fit(X_train, y_train)  
lasso_reg.fit(X_train, y_train)

#save the beta coefficients
beta0_ridge = ridge_reg.intercept_
betas_ridge = ridge_reg.coef_

#save the beta coefficients
beta0_lasso = lasso_reg.intercept_
betas_lasso = lasso_reg.coef_

#make predictions everywhere
ypredict_ridge = ridge_reg.predict(X_train)
ypredict_lasso = lasso_reg.predict(X_train)

print('Ridge Beta0 is:', beta0_ridge)
print('Ridge Betas are:', betas_ridge)

print('Lasso Beta0 is:', beta0_lasso)
print('Lasso Betas are:', betas_lasso)

In [19]:
#--------  sample
# A function to select a random sample of size k from the training set
# Input: 
#      x (n x d array of predictors in training data)
#      y (n x 1 array of response variable vals in training data)
#      k (size of sample) 
# Return: 
#      chosen sample of predictors and responses

def sample(x, y, k):
    n = x.shape[0] # No. of training points
    
    # Choose random indices of size 'k'
    subset_ind = np.random.choice(np.arange(n), k)
    
    # Get predictors and reponses with the indices
    x_subset = x[subset_ind, :]
    y_subset = y[subset_ind]
    
    return (x_subset, y_subset)

In [ ]:
simpl_reg = LinearRegression()
sample_sizes = [100, 150, 200, 250, 300, 350, 400]
columns = ['sample_size', 'avg_training_ridge', 'SD_train_ridge', 'CI_train_ridge', 'avg_test_ridge', 'SD_test_ridge', 
          'CI_test_ridge', 'avg_training_lasso', 'SD_train_lasso', 'CI_train_lasso', 'avg_test_lasso', 'SD_test_lasso', 
          'CI_test_lasso', 'avg_training_simpl', 'SD_train_simpl', 'CI_train_simpl', 'avg_test_simpl', 'SD_test_simpl',
          'CI_test_simpl']
dicts = []

# iterate through sample sizes
for sample_size in sample_sizes:
    
    scores_training_ridge = []
    scores_test_ridge = []
    scores_training_lasso = []
    scores_test_lasso = []
    scores_training_simpl = []
    scores_test_simpl = []
    
    # repeate 10 times for each fit
    for i in range(0, 10):
        sample_X, sample_y = sample(X_train, y_train, sample_size)
        ridge_reg.fit(sample_X, sample_y)  
        lasso_reg.fit(sample_X, sample_y)
        simpl_reg.fit(sample_X, sample_y)
        
        pred_training_ridge = ridge_reg.predict(sample_X)
        pred_test_ridge = ridge_reg.predict(X_test)
        
        pred_training_lasso = lasso_reg.predict(sample_X)
        pred_test_lasso = lasso_reg.predict(X_test)
        
        pred_training_simpl =simpl_reg.predict(sample_X)
        pred_test_simpl = simpl_reg.predict(X_test)
        
        # r2 scores
        score_training_ridge = r2_score(sample_y, pred_training_ridge)
        scores_training_ridge.append(score_training_ridge)
        score_test_ridge = r2_score(y_test, pred_test_ridge)
        scores_test_ridge.append(score_test_ridge)
        
        score_training_lasso = r2_score(sample_y, pred_training_lasso)
        scores_training_lasso.append(score_training_lasso)
        score_test_ridge = r2_score(y_test, pred_test_lasso)
        scores_test_lasso.append(score_test_ridge)
        
        score_training_simpl = r2_score(sample_y, pred_training_simpl)
        scores_training_simpl.append(score_training_simpl)
        score_test_simpl = r2_score(y_test, pred_test_simpl)
        scores_test_simpl.append(score_test_simpl)
        
    # averages over the 10 trials
    avg_test_ridge = np.mean(scores_test_ridge)
    avg_test_lasso = np.mean(scores_test_lasso)
    avg_test_simpl = np.mean(scores_test_simpl)
    avg_training_ridge = np.mean(scores_training_ridge)
    avg_training_lasso = np.mean(scores_training_lasso)
    avg_training_simpl = np.mean(scores_training_simpl)
        
    # compute standard deviations of errors for training and test sets for each regression
    SD_train_ridge = np.std(scores_training_ridge)
    SD_train_lasso = np.std(scores_training_lasso)
    SD_train_simpl = np.std(scores_training_simpl)
    SD_test_ridge = np.std(scores_test_ridge)
    SD_test_lasso = np.std(scores_test_lasso)
    SD_test_simpl = np.std(scores_test_simpl)
    
    # compute confidance intervals
    CI_train_ridge = [avg_training_ridge-SD_train_ridge, avg_training_ridge+SD_train_ridge]
    CI_train_lasso = [avg_training_lasso-SD_train_lasso, avg_training_lasso+SD_train_lasso]
    CI_train_simpl = [avg_training_simpl-SD_train_simpl, avg_training_simpl+SD_train_simpl]
    CI_test_ridge = [avg_test_ridge-SD_train_ridge, avg_test_ridge+SD_test_ridge]
    CI_test_lasso = [avg_test_lasso-SD_train_lasso, avg_test_lasso+SD_test_lasso]
    CI_test_simpl = [avg_test_simpl-SD_train_simpl, avg_test_simpl+SD_test_simpl]
   
   
    
    # create a data structure to store our values
    temp_dict = {'sample_size': sample_size, 'avg_training_ridge' : avg_training_ridge, 'SD_train_ridge' : SD_train_ridge, 'CI_train_ridge' : CI_train_ridge, 'avg_test_ridge' : avg_test_ridge, 
                'CI_test_simpl' : CI_test_simpl, 'CI_test_lasso' : CI_test_lasso, 'CI_test_ridge' : CI_test_ridge, 'CI_train_simpl' : CI_train_simpl, 'CI_train_lasso' :  CI_train_lasso, 'SD_test_lasso' : SD_test_lasso, 'SD_test_ridge' :  SD_test_ridge, 'avg_training_lasso' : avg_training_lasso, 'SD_train_lasso' : SD_train_lasso, 'avg_test_lasso' : avg_test_lasso,
                'avg_training_simpl' : avg_training_simpl, 'SD_train_simpl' : SD_train_simpl, 'avg_test_simpl' : avg_test_simpl, 'SD_test_simpl' : SD_test_simpl} 
    dicts.append(temp_dict)
   
df = pd.DataFrame(dicts) 
df = df[columns]
print(df)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(15,6))


ax1.errorbar(df.sample_size.values, df.avg_training_ridge.values, c='red', yerr=df.SD_train_ridge.values, fmt='o')
ax1.set_title("Ridge Training R2 Scores")
ax1.set_ylabel="R2"
ax1.set_xlabel="sample size"

ax2.errorbar(df.sample_size.values, df.avg_training_lasso.values, c='red', yerr=df.SD_train_lasso.values, fmt='o')
ax2.set_title("Lasso Training R2 Scores")

ax3.errorbar(df.sample_size.values, df.avg_training_simpl.values, c='red', yerr=df.SD_train_simpl.values, fmt='o')
ax3.set_title("Simple Regression Training R2 Scores")

f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(15,6) )
ax1.errorbar(df.sample_size.values, df.avg_test_ridge.values, c='red', yerr=df.SD_test_ridge.values, fmt='o')
ax1.set_title("Ridge Test R2 Scores")

ax2.errorbar(df.sample_size.values, df.avg_test_lasso.values, c='red', yerr=df.SD_test_lasso.values, fmt='o')
ax2.set_title("Lasso Test R2 Scores")

ax3.errorbar(df.sample_size.values, df.avg_test_simpl.values, c='red', yerr=df.SD_test_simpl.values, fmt='o')
ax3.set_title("Simple Regression Test R2 Scores")

plt.show()

- Fit linear, Ridge and Lasso regression models to each of the generated sample. In each case, compute the $R^2$ score for the model on the training sample on which it was fitted, and on the test set.
- Repeat the above experiment for 10 random trials/splits, and compute the average train and test $R^2$ across the trials for each training sample size. Also, compute the standard deviation (SD) in each case.
- Make a plot of the mean training $R^2$ scores for the linear, Ridge and Lasso regression methods as a function of the training sample size. Also, show a confidence interval for the mean scores extending from **mean - SD** to **mean + SD**. Make a similar plot for the test $R^2$ scores.

How do the training and test $R^2$ scores compare for the three methods? Give an explanation for your observations. How do the confidence intervals for the estimated $R^2$ change with training sample size? Based on the plots, which of the three methods would you recommend when one needs to fit a regression model using a small training sample?

*Hint:* You may use `sklearn`'s `RidgeCV` and `LassoCV` classes to implement Ridge and Lasso regression. These classes automatically perform cross-validation to tune the parameter $\lambda$ from a given range of values. You may use the `plt.errorbar` function to plot confidence bars for the average $R^2$ scores.

## Part (g): Polynomial & Interaction Terms

Moving beyond linear models, we will now try to improve the performance of the regression model in Part (b) from HW 3 by including higher-order polynomial and interaction terms. 

- For each continuous predictor $X_j$, include additional polynomial terms $X^2_j$, $X^3_j$, and $X^4_j$, and fit a multiple regression model to the expanded training set. How does the $R^2$ of this model on the test set compare with that of the linear model fitted in Part (b) from HW 3? Using a t-test, find out which of estimated coefficients for the polynomial terms are statistically significant at a significance level of 5%. 

- Fit a multiple linear regression model with additional interaction terms $\mathbb{I}_{month = 12} \times temp$ and $\mathbb{I}_{workingday = 1} \times \mathbb{I}_{weathersit = 1}$ and report the test $R^2$ for the fitted model. How does this compare with the $R^2$ obtained using linear model in Part (b) from HW 3? Are the estimated coefficients for the interaction terms statistically significant at a significance level of 5%?

In [22]:
train_data['temp^2'] = train_data['temp']**2
test_data['temp^2'] = test_data['temp']**2
train_data['temp^3'] = train_data['temp']**3
test_data['temp^3'] = test_data['temp']**3
train_data['temp^4'] = train_data['temp']**4
test_data['temp^4'] = test_data['temp']**4

train_data['atemp^2'] = (train_data['atemp'])**2
test_data['atemp^2'] = (test_data['atemp'])**2
train_data['atemp^3'] = (train_data['atemp'])**3
test_data['atemp^3'] = (test_data['atemp'])**3
train_data['atemp^4'] = (train_data['atemp'])**4
test_data['atemp^4'] = (test_data['atemp'])**4

train_data['humidity^2'] = (train_data['humidity'])**2
test_data['humidity^2'] = (test_data['humidity'])**2
train_data['humidity^3'] = (train_data['humidity'])**3
test_data['humidity^3'] = (test_data['humidity'])**3
train_data['humidity^4'] = (train_data['humidity'])**4
test_data['humidity^4'] = (test_data['humidity'])**4

train_data['windspeed^2'] = (train_data['windspeed'])**2
test_data['windspeed^2'] = (test_data['windspeed'])**2
train_data['windspeed^3'] = (train_data['windspeed'])**3
test_data['windspeed^3'] = (test_data['windspeed'])**3
train_data['windspeed^4'] = (train_data['windspeed'])**4
test_data['windspeed^4'] = (test_data['windspeed'])**4

In [ ]:
#create an array of values for our regression
y_train = train_data['rentals'].values
X_train = train_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4']].values

y_test = test_data['rentals'].values
X_test = test_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4']].values

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

print(X_train.shape, y_train.shape, X_test.shape , y_test.shape)


train_data.head(5)

In [ ]:
# fit model with the additional interaction terms, compute metrics
lm = LinearRegression(fit_intercept=True)
lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)

print('The equation of the regression plane is: {} + {} * x'.format(lm.intercept_, lm.coef_))

train_MSE= np.mean((y_train - lm.predict(X_train))**2)
test_MSE= np.mean((y_test - lm.predict(X_test))**2)
print('The train MSE is {}, the test MSE is {}'.format(train_MSE, test_MSE))

train_R_sq = lm.score(X_train, y_train)
test_R_sq = lm.score(X_test, y_test)
print('The train R^2 is {}, the test R^2 is {}'.format(train_R_sq, test_R_sq))

In [ ]:
# statsmodel regression to easily obtain metrics
# create the X matrix by appending a column of ones to x_train
X = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)
# build the OLS model from the training data
smm = sm.OLS(y_train, X)

#save regression info in results_sm
results_sm = smm.fit()

print(results_sm.summary())
print('Parameters: ', results_sm.params)

In [ ]:
train_data['month12_temp'] = train_data['month_12'] * train_data['temp']

train_data['workday_weather'] = np.where((train_data['workingday'] ==1) & (train_data['weather_2'] ==0) & 
                                (train_data['weather_3']==0), 1, 0)

test_data['month12_temp'] = test_data['month_12'] * test_data['temp']

test_data['workday_weather'] = np.where((test_data['workingday'] ==1) & (test_data['weather_2'] ==0) & 
                                (test_data['weather_3']==0), 1, 0)

In [ ]:
pd.set_option('display.max_columns', None)
train_data.head()

In [ ]:
# create an array of values for our regression
y_train = train_data['rentals'].values
X_train = train_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4', 'workday_weather', 'month12_temp']].values

y_test = test_data['rentals'].values
X_test = test_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4', 'workday_weather', 'month12_temp']].values

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

print(X_train.shape, y_train.shape, X_test.shape , y_test.shape)

train_data.head(5)

In [ ]:
# fit model with the additional interaction terms, compute metrics
lm = LinearRegression(fit_intercept=True)
lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)

print('The equation of the regression plane is: {} + {} * x'.format(lm.intercept_, lm.coef_))

train_MSE= np.mean((y_train - lm.predict(X_train))**2)
test_MSE= np.mean((y_test - lm.predict(X_test))**2)
print('The train MSE is {}, the test MSE is {}'.format(train_MSE, test_MSE))

train_R_sq = lm.score(X_train, y_train)
test_R_sq = lm.score(X_test, y_test)
print('The train R^2 is {}, the test R^2 is {}'.format(train_R_sq, test_R_sq))

In [ ]:
# statsmodel regression to easily obtain metrics

# create the X matrix by appending a column of ones to x_train
X = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)
# build the OLS model from the training data
smm = sm.OLS(y_train, X)

#save regression info in results_sm
results_sm = smm.fit()

print(results_sm.summary())
print('Parameters: ', results_sm.params)

In [ ]:
train_data['month12_temp'] = train_data['month_12'] * train_data['temp']

train_data['workday_weather'] = np.where((train_data['workingday'] ==1) & (train_data['weather_2'] ==0) & 
                                (train_data['weather_3']==0), 1, 0)

test_data['month12_temp'] = test_data['month_12'] * test_data['temp']

test_data['workday_weather'] = np.where((test_data['workingday'] ==1) & (test_data['weather_2'] ==0) & 
                                (test_data['weather_3']==0), 1, 0)

In [ ]:
pd.set_option('display.max_columns', None)
train_data.head()

In [ ]:
# create an array of values for our regression
y_train = train_data['rentals'].values
X_train = train_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4', 'workday_weather', 'month12_temp']].values

y_test = test_data['rentals'].values
X_test = test_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4', 'workday_weather', 'month12_temp']].values

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

print(X_train.shape, y_train.shape, X_test.shape , y_test.shape)

train_data.head(5)

In [ ]:
# fit model with the additional interaction terms, compute metrics
lm = LinearRegression(fit_intercept=True)
lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)

print('The equation of the regression plane is: {} + {} * x'.format(lm.intercept_, lm.coef_))

train_MSE= np.mean((y_train - lm.predict(X_train))**2)
test_MSE= np.mean((y_test - lm.predict(X_test))**2)
print('The train MSE is {}, the test MSE is {}'.format(train_MSE, test_MSE))

train_R_sq = lm.score(X_train, y_train)
test_R_sq = lm.score(X_test, y_test)
print('The train R^2 is {}, the test R^2 is {}'.format(train_R_sq, test_R_sq))

## Part (h): PCA to deal with high dimensionality

We would like to fit a model to include all main effects, polynomial terms up to the $4^{th}$ order, and all interactions between all possible predictors and polynomial terms (not including the interactions between $X^1_j$, $X^2_j$, $X^3_j$, and $X^4_j$ as they would just create higher order polynomial terms).  

- Create an expanded training set including all the desired terms mentioned above.  What are the dimensions of this 'design matrix' of all the predictor variables?   What are the issues with attempting to fit a regression model using all of these predictors?

- Instead of using the usual approaches for model selection, let's instead use principal components analysis (PCA) to fit the model.  First, create the principal component vectors in python (consider: should you normalize first?).  Then fit 5 different regression models: (1) using just the first PCA vector, (2) using the first two PCA vectors, (3) using the first three PCA vectors, etc...  Briefly summarize how these models compare in the training set.

- Use the test set to decide which of the 5 models above is best to predict out of sample.  How does this model compare to the previous models you've fit?  What are the interpretations of this model's coefficients?

In [ ]:
#create polynomial features matrix
polynomial_features = PolynomialFeatures(degree=1, interaction_only=True, include_bias=True)
poly = polynomial_features.fit_transform(X_train)


#Using too many features can create an overfitting problem, particularly if they are of high degree - the training
#process will fit to the noise! Here, the degree of the factors isn't high (limited to 1), but with enough factors,
#you could wind up with more factors than you have data items, in which case the model is not specified.


pca1 = PCA(n_components=1)
pca1.fit(X_train)
X_train_pca1 = pca1.transform(X_train)
X_test_pca1 = pca1.transform(X_test)
print('Explained variance ratio:', pca1.explained_variance_ratio_)

pca2 = PCA(n_components=2)
pca2.fit(X_train)
X_train_pca2 = pca2.transform(X_train)
X_test_pca2 = pca2.transform(X_test)
print('Explained variance ratio:', pca2.explained_variance_ratio_)

pca3 = PCA(n_components=3)
pca3.fit(X_train)
X_train_pca3 = pca3.transform(X_train)
X_test_pca3 = pca3.transform(X_test)
print('Explained variance ratio:', pca3.explained_variance_ratio_)

In [ ]:
regression_model_pca1 = LinearRegression(fit_intercept=True)
regression_model_pca1.fit(X_train_pca1, y_train)
y_pred_1 = regression_model_pca1.predict(X_test_pca1)
score_1 = r2_score(y_test, y_pred_1)

print('PCA w 1 component Test R^2: {}'.format(score_1))

regression_model_pca2 = LinearRegression(fit_intercept=True)
regression_model_pca2.fit(X_train_pca2, y_train)

print('PCA w 2 component Test R^2: {}'.format(regression_model_pca2.score(X_test_pca2, y_test)))

regression_model_pca3 = LinearRegression(fit_intercept=True)
regression_model_pca3.fit(X_train_pca3, y_train)

print('PCA w 3 component Test R^2: {}'.format(regression_model_pca3.score(X_test_pca3, y_test)))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))

ax[0].scatter(X_train_pca3[:, 0], X_train_pca3[:, 1], color='blue', alpha=0.2, label='train R^2')

ax[0].set_title('Dimension Reduced Data')
ax[0].set_xlabel('1st Principal Component')
ax[0].set_ylabel('2nd Principal Component')

ax[1].scatter(X_train_pca3[:, 1], X_train_pca3[:, 2], color='red', alpha=0.2, label='train R^2')

ax[1].set_title('Dimension Reduced Data')
ax[1].set_xlabel('2nd Principal Component')
ax[1].set_ylabel('3rd Principal Component')


plt.show()

## Part (i): Beyond Squared Error

We have seen in class that the multiple linear regression method optimizes the Mean Squared Error (MSE) on the training set. Consider the following alternate evaluation metric, referred to as the Root Mean Squared Logarthmic Error (RMSLE):

$$
\sqrt{\frac{1}{n}\sum_{i=1}^n (log(y_i+1) - log(\hat{y}_i+1))^2}.
$$

The *lower* the RMSLE the *better* is the performance of a model. The RMSLE penalizes errors on smaller responses more heavily than errors on larger responses. For example, the RMSLE penalizes a prediction of $\hat{y} = 15$ for a true response of $y=10$ more heavily than a prediction of $\hat{y} = 105$ for a true response of $100$, though the difference in predicted and true responses are the same in both cases. 

This is a natural evaluation metric for bike share demand prediction, as in this application, it is more important that the prediction model is accurate on days where the demand is low (so that the few customers who arrive are served satisfactorily), compared to days on which the demand is high (when it is less damaging to lose out on some customers).

The following code computes the RMSLE for you:

In [2]:
#--------  rmsle
# A function for evaluating Root Mean Squared Logarithmic Error (RMSLE)
# of the linear regression model on a data set
# Input: 
#      y_test (n x 1 array of response variable vals in testing data)
#      y_pred (n x 1 array of response variable vals in testing data)
# Return: 
#      RMSLE (float) 

def rmsle(y, y_pred):     
    where_are_NaNs = np.isnan(np.log(y_pred+1))
    y_pred[where_are_NaNs] = 0
    rmsle_ = np.sqrt(np.mean(np.square(np.log(y+1) - np.log(y_pred+1))))
    print("RMSLE is ", rmsle_)
    
    return rmsle_

Use the above code to compute the training and test RMSLE for the polynomial regression model you fit in Part (g). 

You are required to develop a strategy to fit a regression model by optimizing the RMSLE on the training set. Give a justification for your proposed approach. Does the model fitted using your approach yield lower train RMSLE than the model in Part (g)? How about the test RMSLE of the new model? 

**Note:** We do not require you to implement a new regression solver for RMSLE. Instead, we ask you to think about ways to use existing built-in functions to fit a model that performs well on RMSLE. Your regression model may use the same polynomial terms used in Part (g).

In [ ]:
print("Train:")
RMSLE = rmsle(y_train, lm.predict(X_train))
print("Test:")
RMSLE = rmsle(y_test, lm.predict(X_test))

## Part (j): Dealing with Erroneous Labels

Due to occasional system crashes, some of the bike counts reported in the data set have been recorded manually. These counts are not very unreliable and are prone to errors. It is known that roughly 5% of the labels in the training set are erroneous (i.e. can be arbitrarily different from the true counts), while all the labels in the test set were confirmed to be accurate. Unfortunately, the identities of the erroneous records in the training set are not available. Can this information about presence of 5% errors in the training set labels (without details about the specific identities of the erroneous rows) be used to improve the performance of the model in Part (g)? Note that we are interested in improving the $R^2$ performance of the model on the test set (not the training $R^2$ score). 

As a final task, we require you to come up with a strategy to fit a regression model, taking into account the errors in the training set labels. Explain the intuition behind your approach (we do not expect a detailed mathematical justification). Use your approach to fit a regression model on the training set, and compare its test $R^2$ with the model in Part (g).

**Note:** Again, we do not require you to implement a new regression solver for handling erroneous labels. It is sufficient that you to come up with an approach that uses existing built-in functions. Your regression model may use the same polynomial terms used in Part (g).

In [ ]:
for x in range(0, 331):
    if (train_data.get_value(x, 'month_2') == 1 or train_data.get_value(x, 'month_3') == 1):
        train_data.set_value('season_2', x, 0)
        train_data.set_value('season_3', x, 0)
        train_data.set_value('season_4', x, 0)
    elif (train_data.get_value(x, 'month_4') == 1 or train_data.get_value(x, 'month_5') == 1 or train_data.get_value(x, 'month_6') == 1):
        train_data.set_value('season_2', x, 1)
        train_data.set_value('season_3', x, 0)
        train_data.set_value('season_4', x, 0)
    elif (train_data.get_value(x, 'month_7') == 1 or train_data.get_value(x, 'month_8') == 1 or train_data.get_value(x, 'month_9') == 1):
        train_data.set_value('season_2', x, 0)
        train_data.set_value('season_3', x, 1)
        train_data.set_value('season_4', x, 0)
    elif (train_data.get_value(x, 'month_10') == 1 or train_data.get_value(x, 'month_11') == 1 or train_data.get_value(x, 'month_12') == 1):
        train_data.set_value('season_2', x, 0)
        train_data.set_value('season_3', x, 0)
        train_data.set_value('season_4', x, 1)
        



y_train = train_data['rentals'].values
y_train = y_train[0:-3]
y_train = y_train.reshape(len(y_train), 1)

X_train = train_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4']].values
X_train = X_train[0:-3]
y_test = test_data['rentals'].values
X_test = test_data[['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_2', 
                 'season_3', 'season_4', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_1', 
                 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'weather_2', 
                 'weather_3', 'temp^2', 'temp^3', 'temp^4', 'atemp^2', 'atemp^3', 'atemp^4', 'humidity^2', 'humidity^3', 
                 'humidity^4', 'windspeed^2', 'windspeed^3', 'windspeed^4']].values

y_test = y_test.reshape(len(y_test), 1)

print(X_train.shape, y_train.shape, X_test.shape , y_test.shape)

lm = LinearRegression(fit_intercept=True)
lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)

print('The equation of the regression plane is: {} + {} * x'.format(lm.intercept_, lm.coef_))

train_MSE= np.mean((y_train - lm.predict(X_train))**2)
test_MSE= np.mean((y_test - lm.predict(X_test))**2)
print('The train MSE is {}, the test MSE is {}'.format(train_MSE, test_MSE))

train_R_sq = lm.score(X_train, y_train)
test_R_sq = lm.score(X_test, y_test)
print('The train R^2 is {}, the test R^2 is {}'.format(train_R_sq, test_R_sq))

In [ ]:
'''
Analysis

1. I would start off with checking the season and month figures for inconsistencies. 
2. I would also check for consistencies between the working day binary parameter and the binary day flags.
3. I would check for sanity in the deltas between atemp and temp

We visually inspected all three, and found a number of errors in 1, i.e., a season number would be ascribed to a
tuple with a month that didn't match. We were only able to find one error for #2, and could not find any errors for #3.
(We visually inspected deltas between temp and atemp and didn't find anything noteworthy.)

Our solution: assign season automatically depending on month for the entire training data set.

It turned out that our r-squared was actually REDUCED from 0.282 to 0.277! Cleaning the data did not impact our
results in any meaningful way. However, it may be that there is some more creative way to clean the data that we
simply did not think of, so we leave the door open to the possibility of obtaining more meaningful results.
'''